<a href="https://colab.research.google.com/github/AnnaK8090/CIND-820_Big-Data-Analytics-Project/blob/main/CIND_820_Big_Data_Analytics_Project_ASSOCIATION_RULES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# installing the apyori package
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [4]:
customersDF = pd.read_csv("olist_customers_dataset.csv")
geolocationDF = pd.read_csv("olist_geolocation_dataset.csv")
order_itemsDF = pd.read_csv("olist_order_items_dataset.csv")
order_paymentsDF = pd.read_csv("olist_order_payments_dataset.csv")
order_reviewsDF = pd.read_csv("olist_order_reviews_dataset.csv")
ordersDF = pd.read_csv("olist_orders_dataset.csv")
productsDF = pd.read_csv("olist_products_dataset.csv")
sellersDF = pd.read_csv("olist_sellers_dataset.csv")
product_category_name_translation = pd.read_csv("product_category_name_translation.csv")

In [5]:
masterDF = ordersDF.copy()
masterDF = masterDF.merge(customersDF,on='customer_id')
masterDF = masterDF.merge(order_reviewsDF,on='order_id')
masterDF = masterDF.merge(order_paymentsDF,on='order_id')
masterDF = masterDF.merge(order_itemsDF,on='order_id')
masterDF = masterDF.merge(productsDF,on='product_id')
masterDF = masterDF.merge(sellersDF,on='seller_id')
masterDF = masterDF.merge(product_category_name_translation,on='product_category_name')


In [6]:
masterDF.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,268.0,4.0,500.0,19.0,8.0,13.0,9350,maua,SP,housewares
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,268.0,4.0,500.0,19.0,8.0,13.0,9350,maua,SP,housewares
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,268.0,4.0,500.0,19.0,8.0,13.0,9350,maua,SP,housewares
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,3a51803cc0d012c3b5dc8b7528cb05f7,3366,...,268.0,4.0,500.0,19.0,8.0,13.0,9350,maua,SP,housewares
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,ef0996a1a279c26e7ecbd737be23d235,2290,...,268.0,4.0,500.0,19.0,8.0,13.0,9350,maua,SP,housewares


In [7]:
masterDF.shape

(115609, 40)

In [8]:
s = masterDF.groupby('order_id').product_category_name_english.nunique()
masterDF = masterDF[masterDF['order_id'].isin(s.index[s>1])]
masterDF.shape

(1943, 40)

In [9]:
masterDF.shape

(1943, 40)

In [10]:
masterDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1943 entries, 53 to 115599
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       1943 non-null   object 
 1   customer_id                    1943 non-null   object 
 2   order_status                   1943 non-null   object 
 3   order_purchase_timestamp       1943 non-null   object 
 4   order_approved_at              1943 non-null   object 
 5   order_delivered_carrier_date   1943 non-null   object 
 6   order_delivered_customer_date  1937 non-null   object 
 7   order_estimated_delivery_date  1943 non-null   object 
 8   customer_unique_id             1943 non-null   object 
 9   customer_zip_code_prefix       1943 non-null   int64  
 10  customer_city                  1943 non-null   object 
 11  customer_state                 1943 non-null   object 
 12  review_id                      1943 non-null 

In [11]:
masterDF["ItemID_NEW_StringType"] = masterDF["product_id"]

In [12]:
masterDF["ItemID_NEW_StringType"] =masterDF["ItemID_NEW_StringType"].astype(str)

In [13]:
masterDF['ItemID_Category'] = [''.join(i) for i in zip(masterDF['ItemID_NEW_StringType'], masterDF['product_category_name_english'])]

In [14]:
masterDF["Quantity"]=1

In [15]:
# Stripping extra spaces in the description
masterDF['ItemID_Category'] = masterDF['ItemID_Category'].str.strip()
  
# Dropping the rows without any invoice number
masterDF.dropna(axis = 0, subset =['order_id'], inplace = True)
masterDF['order_id'] = masterDF['order_id'].astype('str')
  

In [16]:
masterDF_groups =masterDF.groupby('product_category_name_english')
masterDF_groups.groups    


{'air_conditioning': [107223, 107279], 'art': [106213, 106252, 106270, 106318, 106369, 106370, 106372], 'arts_and_craftmanship': [114791, 114801], 'audio': [63893, 63920, 63951, 63985, 64043, 64224], 'auto': [54078, 54095, 54144, 54153, 54463, 54488, 54552, 54553, 54566, 54619, 54765, 54807, 54810, 54813, 55105, 55358, 55841, 55842, 55937, 56330, 56423, 56451, 56638, 56835, 56841, 56845, 56895, 56933, 56940, 57140, 57176, 57283, 57326, 57386, 57387, 57485, 57486, 57487, 57490, 57575, 57576, 57614], 'baby': [7414, 7570, 7573, 7688, 7689, 7695, 7696, 7702, 7725, 7746, 7757, 7765, 7772, 7853, 7912, 7913, 7914, 7949, 8028, 8043, 8134, 8135, 8136, 8152, 8153, 8174, 8175, 8183, 8200, 8201, 8202, 8211, 8215, 8216, 8220, 8286, 8294, 8295, 8315, 8332, 8337, 8396, 8450, 8466, 8556, 8596, 8597, 8598, 8604, 8609, 8610, 8613, 8616, 8619, 8621, 8622, 8626, 8627, 8629, 8633, 8647, 8648, 8649, 8650, 8651, 8697, 8698, 8718, 8719, 8757, 8765, 8821, 8889, 8893, 8898, 8900, 8902, 8903, 8911, 8927, 8937, 9

In [17]:
masterDF_groups.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,product_category_name_english,ItemID_NEW_StringType,ItemID_Category,Quantity
53,591083bc42b589c7052118aa83118e76,276df8efcf9602ecddaa00e349eb79b3,delivered,2018-01-02 22:09:17,2018-01-02 22:19:30,2018-01-03 21:10:00,2018-01-23 13:54:57,2018-02-06 00:00:00,94bd5dee83e9590368179f4ab026b4f4,95097,...,30.0,15.0,15.0,88301,itajai,SP,housewares,19944bc70a13c9b62e18a2bc7e913bb6,19944bc70a13c9b62e18a2bc7e913bb6housewares,1
54,591083bc42b589c7052118aa83118e76,276df8efcf9602ecddaa00e349eb79b3,delivered,2018-01-02 22:09:17,2018-01-02 22:19:30,2018-01-03 21:10:00,2018-01-23 13:54:57,2018-02-06 00:00:00,94bd5dee83e9590368179f4ab026b4f4,95097,...,30.0,15.0,15.0,88301,itajai,SP,housewares,19944bc70a13c9b62e18a2bc7e913bb6,19944bc70a13c9b62e18a2bc7e913bb6housewares,1
55,591083bc42b589c7052118aa83118e76,276df8efcf9602ecddaa00e349eb79b3,delivered,2018-01-02 22:09:17,2018-01-02 22:19:30,2018-01-03 21:10:00,2018-01-23 13:54:57,2018-02-06 00:00:00,94bd5dee83e9590368179f4ab026b4f4,95097,...,30.0,15.0,15.0,88301,itajai,SP,housewares,19944bc70a13c9b62e18a2bc7e913bb6,19944bc70a13c9b62e18a2bc7e913bb6housewares,1
56,591083bc42b589c7052118aa83118e76,276df8efcf9602ecddaa00e349eb79b3,delivered,2018-01-02 22:09:17,2018-01-02 22:19:30,2018-01-03 21:10:00,2018-01-23 13:54:57,2018-02-06 00:00:00,94bd5dee83e9590368179f4ab026b4f4,95097,...,30.0,15.0,15.0,88301,itajai,SP,housewares,19944bc70a13c9b62e18a2bc7e913bb6,19944bc70a13c9b62e18a2bc7e913bb6housewares,1
57,591083bc42b589c7052118aa83118e76,276df8efcf9602ecddaa00e349eb79b3,delivered,2018-01-02 22:09:17,2018-01-02 22:19:30,2018-01-03 21:10:00,2018-01-23 13:54:57,2018-02-06 00:00:00,94bd5dee83e9590368179f4ab026b4f4,95097,...,30.0,15.0,15.0,88301,itajai,SP,housewares,19944bc70a13c9b62e18a2bc7e913bb6,19944bc70a13c9b62e18a2bc7e913bb6housewares,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115397,be0f8b9e1444dc69d1288a37e32d2895,4471480a2199c933b05f842342a6c57a,delivered,2018-06-11 00:23:42,2018-06-11 00:35:12,2018-06-11 14:41:00,2018-06-15 19:38:36,2018-07-12 00:00:00,859f40a26e8b81608c6d035211098e9b,5640,...,60.0,10.0,40.0,85904,toledo,PR,furniture_bedroom,d373a9b9079c251371e94cacf40c1fe7,d373a9b9079c251371e94cacf40c1fe7furniture_bedroom,1
115441,d1175caace64414de4ca9404ab747e2e,f0bf95715c175ad8e39251a421a76255,delivered,2018-03-16 23:37:45,2018-03-16 23:50:20,2018-03-19 22:14:26,2018-03-27 17:04:59,2018-04-05 00:00:00,99eb56d3a7b9189c232c5e8f7d85b982,37701,...,45.0,12.0,12.0,9780,sao bernardo do campo,SP,cine_photo,071b5dd29feece041c356d172474b113,071b5dd29feece041c356d172474b113cine_photo,1
115452,91be51c856a90d7efe86cf9d082d6ae3,637321037fb8b34323ce7fd8aab4a0f1,delivered,2018-06-12 15:58:03,2018-06-12 16:32:26,2018-06-12 15:44:00,2018-06-15 15:42:07,2018-07-12 00:00:00,07547425c8f7a7086f529f2d3b80d21a,14806,...,23.0,21.0,21.0,7032,guarulhos,SP,cine_photo,96627b65a0070635b620d07f97e63347,96627b65a0070635b620d07f97e63347cine_photo,1
115476,91be51c856a90d7efe86cf9d082d6ae3,637321037fb8b34323ce7fd8aab4a0f1,delivered,2018-06-12 15:58:03,2018-06-12 16:32:26,2018-06-12 15:44:00,2018-06-15 15:42:07,2018-07-12 00:00:00,07547425c8f7a7086f529f2d3b80d21a,14806,...,24.0,4.0,12.0,9041,santo andre,SP,cine_photo,73bcea4312cb31d72bffc6562f7f7c71,73bcea4312cb31d72bffc6562f7f7c71cine_photo,1


In [18]:
masterDF_groups.groups

{'air_conditioning': [107223, 107279], 'art': [106213, 106252, 106270, 106318, 106369, 106370, 106372], 'arts_and_craftmanship': [114791, 114801], 'audio': [63893, 63920, 63951, 63985, 64043, 64224], 'auto': [54078, 54095, 54144, 54153, 54463, 54488, 54552, 54553, 54566, 54619, 54765, 54807, 54810, 54813, 55105, 55358, 55841, 55842, 55937, 56330, 56423, 56451, 56638, 56835, 56841, 56845, 56895, 56933, 56940, 57140, 57176, 57283, 57326, 57386, 57387, 57485, 57486, 57487, 57490, 57575, 57576, 57614], 'baby': [7414, 7570, 7573, 7688, 7689, 7695, 7696, 7702, 7725, 7746, 7757, 7765, 7772, 7853, 7912, 7913, 7914, 7949, 8028, 8043, 8134, 8135, 8136, 8152, 8153, 8174, 8175, 8183, 8200, 8201, 8202, 8211, 8215, 8216, 8220, 8286, 8294, 8295, 8315, 8332, 8337, 8396, 8450, 8466, 8556, 8596, 8597, 8598, 8604, 8609, 8610, 8613, 8616, 8619, 8621, 8622, 8626, 8627, 8629, 8633, 8647, 8648, 8649, 8650, 8651, 8697, 8698, 8718, 8719, 8757, 8765, 8821, 8889, 8893, 8898, 8900, 8902, 8903, 8911, 8927, 8937, 9

In [19]:
#options = ["bed_bath_table","baby","furniture_decor","pet_shop","sports_leisure","auto","fashion_bags_accessories"]

In [20]:
#masterDF = masterDF[masterDF['product_category_name_english'].isin(options)]


In [21]:
basket_bed_bath_table = (masterDF
          .groupby(['order_id', 'ItemID_Category'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('order_id'))


In [22]:
basket_bed_bath_table.head()

ItemID_Category,0042f1a9a7e0edd1400c6cd0fda065f8health_beauty,007c63ae4b346920756b5adcad8095dehousewares,00ba6d766f0b1d7b78a5ce3e1e033263housewares,011377a7487fef47fc9e73fa5f7322a6baby,014a8a503291921f7b004a5215bb3c36baby,01b660ebc1a0c293ccf9b117fa6dd8bbhousewares,01c2e91674406ebaca6a1bbf7f61c3f0computers_accessories,01cf7c4cffff8db0a1cbe612bd2d50a4furniture_decor,01e20e6604216c8adb31d463214ba00chome_construction,01fc56750f0d3444c4a1746ecf19dee0sports_leisure,...,fe9c4b2cf9c3adbaf3644880fb72254dfurniture_decor,fe9dfbe7f974621789683b7b78be2a16health_beauty,feb4ade62e32b8d74c6f69f635057964furniture_living_room,fec2d939a171210847d8f2d102f0dba5garden_tools,fec3b45dc09b257690a09a742870b149garden_tools,fef0296e6442db59700c92c692c90e90costruction_tools_garden,fef7934cc233ee5b1dc13094d98a1465cool_stuff,ff7263dfb3cfff5421ada48c3899c313construction_tools_construction,ff85ff517698c3fe8b200afddda7fb3ahousewares,ffbc83054b3741a8d67fc59d9cf9d42dhousewares
order_id,,,,,,,,,,,,,,,,,,,,,
002f98c0f7efd42638ed6100ca699b42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
005d9a5423d47281ac463a968b3936fb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
014405982914c2cde2796ddcf0b8703d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01b1a7fdae9ad1837d6ab861705a1fa5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01cce1175ac3c4a450e3a0f856d02734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Hot encoding to make the data suitable for Apriori library
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1
  
# Encoding the datasets
masterDF_encoded = basket_bed_bath_table.applymap(hot_encode)
basket_bed_bath_table = masterDF_encoded

In [24]:
# Building the model
frq_items = apriori(basket_bed_bath_table, min_support = 0.001, use_colnames = True)
  
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                                         antecedents  \
0    (0042f1a9a7e0edd1400c6cd0fda065f8health_beauty)   
1        (3e45fc327c9740f1ae03383a8e201968perfumery)   
2  (38add59349dc5b9c3effc6b93925cb97furniture_decor)   
3       (007c63ae4b346920756b5adcad8095dehousewares)   
4     (b931645cdc2d9868f01544e8db63f5abgarden_tools)   

                                         consequents  antecedent support  \
0        (3e45fc327c9740f1ae03383a8e201968perfumery)            0.001385   
1    (0042f1a9a7e0edd1400c6cd0fda065f8health_beauty)            0.001385   
2       (007c63ae4b346920756b5adcad8095dehousewares)            0.001385   
3  (38add59349dc5b9c3effc6b93925cb97furniture_decor)            0.001385   
4       (00ba6d766f0b1d7b78a5ce3e1e033263housewares)            0.001385   

   consequent support   support  confidence   lift  leverage  conviction  
0            0.001385  0.001385         1.0  722.0  0.001383         inf  
1            0.001385  0.001385         1.0  722.0  0.00

In [27]:
print(rules.head(150))

                                           antecedents  \
0      (0042f1a9a7e0edd1400c6cd0fda065f8health_beauty)   
1          (3e45fc327c9740f1ae03383a8e201968perfumery)   
2    (38add59349dc5b9c3effc6b93925cb97furniture_decor)   
3         (007c63ae4b346920756b5adcad8095dehousewares)   
4       (b931645cdc2d9868f01544e8db63f5abgarden_tools)   
..                                                 ...   
245       (7cefa61c9500d9b227894dc48d8b5a0bhousewares)   
246       (0e14941c183946f60e3b86585d05f984housewares)   
247   (873eb5f3b8cc503730e472a14cd26616bed_bath_table)   
250  (5a286d54a7400c4a0921c6e99300d684computers_acc...   
251    (0e34466be9b2d3af8e189b6d27d5eaf3health_beauty)   

                                           consequents  antecedent support  \
0          (3e45fc327c9740f1ae03383a8e201968perfumery)            0.001385   
1      (0042f1a9a7e0edd1400c6cd0fda065f8health_beauty)            0.001385   
2         (007c63ae4b346920756b5adcad8095dehousewares)            0.0